In [32]:
from fastai.vision import *

In [33]:
# TODO properties
model = load_learner('resources/', 'model.pkl')

In [42]:
# TODO properties
path_images = "images"
path_predictions = "images_predicted"

for i in os.listdir(path_images):
    if i.endswith('.jpg'): # TODO png?
        img = open_image(os.path.join(path_images, i))
        img_title, img_extension = os.path.splitext(i)
        pred_class, pred_idx, outputs = model.predict(img)
        prediction = (str(pred_class), round(max(outputs.numpy()) * 100))
        
        path_to_class = path_predictions + "/" + str(pred_class)
        # TODO find all categories from model prehand..
        if not os.path.exists(path_to_class):
            os.makedirs(path_to_class)
        
        new_img_title = img_title + "_" + prediction[0] + "_" + str(prediction[1]) + img_extension
        new_img_path = os.path.join(path_to_class, new_img_title)
        img.save(new_img_path)